# 🧠 Wstęp do biblioteki PyTorch — podstawy potrzebne do zbudowanie prostej sieci neuronowej


## Do czego służy PyTorch?

PyTorch jest bardzo szeroko używaną biblioteką, szczególnie do zadań takich jak:

- Deep learning
- Computer vision and NLP
- Uczenie ze wzmocnieniem

### Mocne strony:
- Automatyzacja wielu procesów związanych z budowaniem i uczeniem sieci neuronowych
- Oparta na języku Python- prosta w użyciu
- Wsparcie GPU


In [35]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

print("PyTorch version:", torch.__version__)


PyTorch version: 2.5.1


## Podstawowy typ danych- tensor

In [36]:

# Tworzenie tensorów
x = torch.tensor([1.0, 2.0, 3.0])
y = torch.ones(3)
z = torch.rand(3)

print(x)
print(y)
print(z)
print()

# Możemy wykonywać na nich operacje jak na macierzach
print("Dodawanie:", x + y)
print("Dot product:", torch.dot(x, y))

# Zmiana wymiarów
a = torch.randn(2, 3)
print("Orginalne wymiary:", a.shape)
print("Zmiana wymiaru na (3,2):", a.view(3, 2))


tensor([1., 2., 3.])
tensor([1., 1., 1.])
tensor([0.9872, 0.2583, 0.0940])

Dodawanie: tensor([2., 3., 4.])
Dot product: tensor(6.)
Orginalne wymiary: torch.Size([2, 3])
Zmiana wymiaru na (3,2): tensor([[-0.4530, -1.3707],
        [ 0.2331, -0.2800],
        [ 0.8277,  0.0197]])


## Automatyczne różniczkowanie

In [37]:

x = torch.tensor([3.0], requires_grad=True)
y = x ** 2
y.backward()
print("Pochodna y względem x:", x.grad)


Pochodna y względem x: tensor([6.])


## Nasza pierwsza sieć neuronowa

In [38]:

X = torch.tensor([[0.,0.],[0.,1.],[1.,0.],[1.,1.]])
y = torch.tensor([[0.],[1.],[1.],[0.]])

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(2, 4)
        self.fc2 = nn.Linear(4, 1)

        torch.nn.init.kaiming_uniform_(self.fc1.weight, nonlinearity='sigmoid')
        torch.nn.init.kaiming_uniform_(self.fc2.weight, nonlinearity='sigmoid')

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

model = SimpleNN()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(10000):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')


Epoch 0, Loss: 0.3014
Epoch 1000, Loss: 0.2489
Epoch 2000, Loss: 0.2448
Epoch 3000, Loss: 0.2297
Epoch 4000, Loss: 0.1844
Epoch 5000, Loss: 0.1039
Epoch 6000, Loss: 0.0451
Epoch 7000, Loss: 0.0232
Epoch 8000, Loss: 0.0144
Epoch 9000, Loss: 0.0101


## Ewaluacja modelu

In [39]:

with torch.no_grad():
    predictions = model(X)
    print("Predictions:")
    print(predictions.round())


Predictions:
tensor([[0.],
        [1.],
        [1.],
        [0.]])


## Klasyfikacja Irysów

In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Ładujemy dane i odpowiednio je dzielimy

In [41]:
# Ładowanie danych
iris = load_iris()
X = iris.data
y = iris.target

# Normalizacja
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dzielimy na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Zamieniamy typ danych na obsługiwany przez pytorcha- tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

## DataLoader- to on będzie pozwalał nam wygodnie wielokrotnie przechodzić przez dane treningowe i testowe

In [42]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


## Szkielet naszej sieci neuronowej

In [43]:
class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(4, 16)
        self.fc2 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = IrisNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

## Trening!

In [44]:
epochs = 100
for epoch in range(epochs):
    total_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

Epoch 10/100, Loss: 2.0516
Epoch 20/100, Loss: 0.7308
Epoch 30/100, Loss: 0.5096
Epoch 40/100, Loss: 0.4862
Epoch 50/100, Loss: 0.3951
Epoch 60/100, Loss: 0.4455
Epoch 70/100, Loss: 0.4814
Epoch 80/100, Loss: 0.3514
Epoch 90/100, Loss: 0.4179
Epoch 100/100, Loss: 0.3974


## Sprawdzamy, jak mu poszło

In [45]:
correct = 0
total = 0
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        _, predicted = torch.max(outputs.data, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

accuracy = 100 * correct / total
print(f"Iris Classification Accuracy: {accuracy:.2f}%")

Iris Classification Accuracy: 100.00%
